# Init Notebook

In [13]:
import asyncio
import itertools
import logging
import random
import nest_asyncio
nest_asyncio.apply()# must be run at the start of every notebook to patch the event loop to allow nesting, eg. so we can run asyncio.run from within a notebook.

In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_107/2324106270.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [15]:
import os
from pathlib import Path
import tempfile

import hydra

# Training Config

In [ ]:
%pwd

In [ ]:
%ls

In [20]:
# Location of path with all training configs

CONFIG_PATH = '../nuplan/planning/script/config/training'
CONFIG_NAME = 'default_training'

# Create a temporary directory to store the cache and experiment artifacts
SAVE_DIR = Path('/home/ehdykhne/nuplan-devkit/henry_experiments') / 'run_sim_closed_loop'  # optionally replace with persistent dir
EXPERIMENT = 'training_raster_experiment'
JOB_NAME = 'train_default_raster'
LOG_DIR = SAVE_DIR / EXPERIMENT / JOB_NAME

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'group={str(SAVE_DIR)}',
    f'cache.cache_path={str(SAVE_DIR)}/cache',
    f'experiment_name={EXPERIMENT}',
    f'job_name={JOB_NAME}',
    'py_func=train',
    '+training=training_raster_model',  # raster model that consumes ego, agents and map raster layers and regresses the ego's trajectory
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_filter.limit_total_scenarios=500',  # Choose 500 scenarios to train with
    'lightning.trainer.params.accelerator=ddp_spawn',  # ddp is not allowed in interactive environment, using ddp_spawn instead - this can bottleneck the data pipeline, it is recommended to run training outside the notebook
    'lightning.trainer.params.max_epochs=10',
    'data_loader.params.batch_size=8',
    'data_loader.params.num_workers=8',
])
print(SAVE_DIR)
print(LOG_DIR)


/home/ehdykhne/nuplan-devkit/henry_experiments/run_sim_closed_loop
/home/ehdykhne/nuplan-devkit/henry_experiments/run_sim_closed_loop/training_raster_experiment/train_default_raster


# Train Model

In [ ]:
from nuplan.planning.script.run_training import main as main_train

# Run the training loop, optionally inspect training artifacts through tensorboard (above cell)
main_train(cfg)

## Prepare the simulation config

In [31]:


last_experiment = sorted(os.listdir(LOG_DIR))[-1]
train_experiment_dir = sorted(LOG_DIR.iterdir())[-1]
checkpoint = sorted((train_experiment_dir / 'checkpoints').iterdir())[-1]

MODEL_PATH = str(checkpoint).replace("=", "\=")


# Location of path with all simulation configs
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation'

# Select the planner and simulation challenge
PLANNER = 'ml_planner'  # [simple_planner, ml_planner]
OBSERVATION = 'ego_centric_ml_agents_observation'#'idm_agents_observation'
CHALLENGE = 'closed_loop_reactive_agents'  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents]
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_filter=all_scenarios',  # initially select all scenarios in the database
    'scenario_filter.scenario_types=[near_multiple_vehicles, starting_unprotected_cross_turn, high_magnitude_jerk]',  # select scenario types
    'scenario_filter.num_scenarios_per_type=2',  # use 2 scenarios per scenario type
]

# Name of the experiment
EXPERIMENT = 'simulation_simple_experiment'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',
    f'group={SAVE_DIR}',    
    f'planner={PLANNER}',
    f'model=raster_model',
    'planner.ml_planner.model_config=${model}',  # hydra notation to select model config
    f'planner.ml_planner.checkpoint_path={MODEL_PATH}',  # this path can be replaced by the checkpoint of the model trained in the previous section
    f'observation={OBSERVATION}',
    'observation.model_config=${model}',
    f'observation.checkpoint_path={MODEL_PATH}',
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
])

#   - override /observation: idm_agents_observation
#   - override /ego_controller: two_stage_controller
#   - override /planner: simple_planner
#   - override /simulation_metric: simulation_closed_loop_reactive_agents

# Simulate in closed loop with trained model controlling ALL agents

In [32]:
from nuplan.planning.script.run_simulation import main as main_simulation

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg)

# Simple simulation folder for visualization in nuBoard
simple_simulation_folder = cfg.output_dir

Global seed set to 0


2023-11-20 09:40:29,029 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2023-11-20 09:40:29,029 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2023-11-20 09:40:29,063 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2023-11-20 09:40:29,063 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2023-11-20 09:40:29,334 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-20 09:40:29,334 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-20 09:40:29,336 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/

2023-11-20 09:40:33,166	INFO worker.py:1673 -- Started a local Ray instance.


2023-11-20 09:40:35,942 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2023-11-20 09:40:35,942 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2023-11-20 09:40:35,943 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 1
Number of threads across all nodes: 32
2023-11-20 09:40:35,943 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 1
Number of threads across all nodes: 32
2023-11-20 09:40:35,943 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-11-20 09:40:35,943 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py

Ray objects: 100%|██████████| 32/32 [00:38<00:00,  1.19s/it]


...

Ray objects:   0%|          | 0/6 [00:00<?, ?it/s](wrapped_fn pid=13942) WARNING:nuplan.planning.simulation.runner.executor:----------- Simulation failed: with the following trace:
(wrapped_fn pid=13942) Traceback (most recent call last):
(wrapped_fn pid=13942)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py", line 27, in run_simulation
(wrapped_fn pid=13942)     return sim_runner.run()
(wrapped_fn pid=13942)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/simulations_runner.py", line 117, in run
(wrapped_fn pid=13942)     self.simulation.propagate(trajectory)
(wrapped_fn pid=13942)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/simulation.py", line 172, in propagate
(wrapped_fn pid=13942)     self._observations.update_observation(iteration, next_iteration, self._history_buffer)
(wrapped_fn pid=13942)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/observation/abstract_ml_agents.py", line 104, in

Ray objects: 100%|██████████| 6/6 [01:17<00:00, 12.91s/it]


2023-11-20 09:43:05,789 WARNING {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py:123}  Failed Simulation.
 'Traceback (most recent call last):
  File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py", line 27, in run_simulation
    return sim_runner.run()
  File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/simulations_runner.py", line 117, in run
    self.simulation.propagate(trajectory)
  File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/simulation.py", line 172, in propagate
    self._observations.update_observation(iteration, next_iteration, self._history_buffer)
  File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/observation/abstract_ml_agents.py", line 104, in update_observation
    predictions = self._infer_model(features)
  File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/observation/ego_centric_ml_agents.py", line 72, in _infer_model
    raise ValueError(f"Prediction do

Rendering histograms: 0it [00:00, ?it/s]

2023-11-20 09:43:05,813 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_summary_callback.py:344}  Metric summary: 00:00:00 [HH:MM:SS]
2023-11-20 09:43:05,813 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/main_callback/metric_summary_callback.py:344}  Metric summary: 00:00:00 [HH:MM:SS]
2023-11-20 09:43:05,814 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:80}  Finished running simulation!
2023-11-20 09:43:05,814 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:80}  Finished running simulation!


# Run Sim with pretrained planner and normal closed loop reactive agents.

In [ ]:
# SPLIT=val14_split
# CHALLENGE=closed_loop_reactive_agents # open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents
# CHECKPOINT_PATH="/path/to/checkpoint.ckpt"

# python $NUPLAN_DEVKIT_ROOT/nuplan/planning/script/run_simulation.py \
# +simulation=$CHALLENGE \
# planner=ml_planner \
# scenario_filter=$SPLIT \
# scenario_builder=nuplan \
# planner.ml_planner.model_config='\${model}' \
# planner.ml_planner.checkpoint_path=$CHECKPOINT_PATH \
# model=urban_driver_open_loop_model \
# hydra.searchpath="[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]"

In [27]:
# Location of path with all simulation configs
CONFIG_PATH = '../nuplan/planning/script/config/simulation'
CONFIG_NAME = 'default_simulation'

CHECKPOINT_PATH='pretrained_checkpoints/gc_pgp_checkpoint.ckpt'

# Select the planner and simulation challenge
PLANNER = 'ml_planner'  # [simple_planner, ml_planner]
CHALLENGE = 'closed_loop_reactive_agents'  # [open_loop_boxes, closed_loop_nonreactive_agents, closed_loop_reactive_agents]
DATASET_PARAMS = [
    'scenario_builder=nuplan_mini',  # use nuplan mini database
    'scenario_filter=all_scenarios',  # initially select all scenarios in the database
    'scenario_filter.scenario_types=[near_multiple_vehicles, on_pickup_dropoff, starting_unprotected_cross_turn, high_magnitude_jerk]',  # select scenario types
    'scenario_filter.num_scenarios_per_type=2',  # use 2 scenarios per scenario type
]

# Name of the experiment
EXPERIMENT = 'simulation_simple_experiment'

# Initialize configuration management system
hydra.core.global_hydra.GlobalHydra.instance().clear()  # reinitialize hydra if already initialized
hydra.initialize(config_path=CONFIG_PATH)

# Compose the configuration
cfg = hydra.compose(config_name=CONFIG_NAME, overrides=[
    f'experiment_name={EXPERIMENT}',
    f'planner={PLANNER}',
    f'model=gc_pgp_model',
    'planner.ml_planner.model_config=${model}',  # hydra notation to select model config
    f'planner.ml_planner.checkpoint_path={CHECKPOINT_PATH}',  # this path can be replaced by the checkpoint of the model trained in the previous section
#     f'observation={OBSERVATION}',
#     'observation.model_config=${model}',
#     f'observation.checkpoint_path={CHECKPOINT_PATH}',
    f'group={SAVE_DIR}',
    f'+simulation={CHALLENGE}',
    *DATASET_PARAMS,
    'hydra.searchpath=[pkg://tuplan_garage.planning.script.config.common, pkg://tuplan_garage.planning.script.config.simulation, pkg://nuplan.planning.script.config.common, pkg://nuplan.planning.script.experiments]'
])

# Simulate in closed loop with pretrained checkpoint for planner

In [28]:
from nuplan.planning.script.run_simulation import main as main_simulation

# Run the simulation loop (real-time visualization not yet supported, see next section for visualization)
main_simulation(cfg)

# Simple simulation folder for visualization in nuBoard
simple_simulation_folder = cfg.output_dir

Global seed set to 0


2023-11-20 09:36:22,580 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2023-11-20 09:36:22,580 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:20}  Building MultiMainCallback...
2023-11-20 09:36:23,383 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2023-11-20 09:36:23,383 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/main_callback_builder.py:35}  Building MultiMainCallback: 4...DONE!
2023-11-20 09:36:23,892 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-20 09:36:23,892 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:19}  Building WorkerPool...
2023-11-20 09:36:23,895 WARNING {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreadi

2023-11-20 09:36:32,409	INFO worker.py:1673 -- Started a local Ray instance.


2023-11-20 09:36:36,199 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2023-11-20 09:36:36,199 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:101}  Worker: RayDistributed
2023-11-20 09:36:36,200 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 1
Number of threads across all nodes: 32
2023-11-20 09:36:36,200 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/utils/multithreading/worker_pool.py:102}  Number of nodes: 1
Number of CPUs per node: 32
Number of GPUs per node: 1
Number of threads across all nodes: 32
2023-11-20 09:36:36,201 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py:27}  Building WorkerPool...DONE!
2023-11-20 09:36:36,201 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/worker_pool_builder.py

Ray objects: 100%|██████████| 32/32 [00:44<00:00,  1.40s/it]


2023-11-20 09:37:21,135 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:74}  Building metric engines...
2023-11-20 09:37:21,135 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:74}  Building metric engines...
2023-11-20 09:37:21,300 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:76}  Building metric engines...DONE
2023-11-20 09:37:21,300 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:76}  Building metric engines...DONE
2023-11-20 09:37:21,300 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:80}  Building simulations from 7 scenarios...
2023-11-20 09:37:21,300 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:80}  Building simulations from 7 scenarios...
2023-11-20 09:37:21,313 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/mod

2023-11-20 09:37:29,659 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:18}  Building TorchModuleWrapper...
2023-11-20 09:37:29,659 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:18}  Building TorchModuleWrapper...
2023-11-20 09:37:29,672 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:21}  Building TorchModuleWrapper...DONE!
2023-11-20 09:37:29,672 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/model_builder.py:21}  Building TorchModuleWrapper...DONE!
2023-11-20 09:37:29,830 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:133}  Building simulations...DONE!
2023-11-20 09:37:29,830 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/builders/simulation_builder.py:133}  Building simulations...DONE!
2023-11-20 09:37:29,830 INFO {/home/ehdykhne/nuplan-devkit/nuplan/planning/script/run_simulation.py:78}  Running simul

Ray objects:  14%|█▍        | 1/7 [00:48<04:53, 48.90s/it](wrapped_fn pid=11801) WARNING:nuplan.planning.simulation.runner.executor:----------- Simulation failed: with the following trace:
(wrapped_fn pid=11801) Traceback (most recent call last):
(wrapped_fn pid=11801)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/executor.py", line 27, in run_simulation
(wrapped_fn pid=11801)     return sim_runner.run()
(wrapped_fn pid=11801)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/runner/simulations_runner.py", line 117, in run
(wrapped_fn pid=11801)     self.simulation.propagate(trajectory)
(wrapped_fn pid=11801)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/simulation.py", line 172, in propagate
(wrapped_fn pid=11801)     self._observations.update_observation(iteration, next_iteration, self._history_buffer)
(wrapped_fn pid=11801)   File "/home/ehdykhne/nuplan-devkit/nuplan/planning/simulation/observation/abstract_ml_agents.py", line

KeyboardInterrupt: 